In [2]:
import torch
import torch.nn as nn 
import os 
import sys
project_root = os.path.abspath("..")  # Adjust if needed

# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.attentionVae import AttentionVAE
from proteinshake.datasets import ProteinLigandInterfaceDataset, AlphaFoldDataset, GeneOntologyDataset
from src.utils import data_utils as dtu
from src.dataset_classes.sequenceDataset import SequenceDataset
from torch.utils.data import DataLoader, Dataset, Subset
from src.utils.data_utils import *

%load_ext autoreload
%autoreload 2


In [3]:
dataset = ProteinLigandInterfaceDataset(root='../data').to_point().torch()

In [4]:
max_seq_length = 500
seq_dataset = SequenceDataset(dataset, max_seq_length, transformer_input=True, return_proteins = True)
seq_dataloader = DataLoader(seq_dataset, batch_size = 128, shuffle = False)

100%|██████████| 4642/4642 [00:01<00:00, 2615.67it/s]


In [5]:
torch.sum(seq_dataset.__getitem__(0)[1], dim = 1)[-84:]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [6]:
next(iter(seq_dataloader))[1].shape

torch.Size([128, 500, 21])

In [28]:
N_layers =  1
embed_dim = 256
hidden_dim = 256
num_heads = 2
dropout = 0.1
latent_dim = 2
seq_len = 500
if torch.cuda.is_available():
    torch.cuda.current_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

attention_vae_test = AttentionVAE(optimizer=torch.optim.Adam, 
                                  optimizer_param={'lr':0.001},
                                  embed_dim=embed_dim,
                                  hidden_dim=hidden_dim,
                                  num_heads=num_heads,
                                  dropout=dropout,
                                  latent_dim=latent_dim,
                                  seq_len=seq_len)

In [9]:
test_in = next(iter(seq_dataloader))

In [112]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [113]:
attention_vae_test.get_attention_weights = True
attention_vae_test.pos_encoder.device = device

In [114]:
device

device(type='cpu')

In [12]:
test_out = attention_vae_test.training_step(test_in,0)

/opt/anaconda3/envs/ProteinManifoldLearning/lib/python3.10/site-packages/pytorch_lightning/core/module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


In [ ]:
attention_vae_test

{}

In [93]:
test_out[4].shape

torch.Size([128, 500, 21])

In [50]:
amino_encoding_dict = {'S': 0,'Y': 1,'F': 2,'K': 3,'V': 4,'C': 5,
                            'A': 6,'H': 7,'T': 8,'I': 9,'W': 10,'R': 11,
                            'P': 12,'L': 13,'G': 14,'N': 15,'E': 16,
                            'Q': 17,'M': 18,'D': 19, 'X':20}

In [44]:
k = next(iter(seq_dataloader))

In [23]:
k

NameError: name 'k' is not defined

In [51]:
embedding_layer = nn.Embedding(21,256, padding_idx=20)

In [72]:
sum(k[0] == 20)

tensor(84)

In [75]:
embedding_layer(k).shape

torch.Size([128, 500, 256])

In [32]:
attention_vae_test.attention_weights[0].shape

torch.Size([128, 500, 500])

In [29]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
optimizer = torch.optim.Adam
optimizer_param = {'lr':0.001}
trainer = pl.Trainer(max_epochs=100,
    accelerator="auto",
    devices=1,
    logger=TensorBoardLogger(save_dir="logs/"))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [30]:
trainer.fit(attention_vae_test, seq_dataloader)


  | Name                        | Type                    | Params | Mode 
--------------------------------------------------------------------------------
0 | dropout_layer               | Dropout                 | 0      | train
1 | embedding_layer             | Embedding               | 5.4 K  | train
2 | pos_encoder                 | PositionalEncoding      | 0      | train
3 | transformer_encoder         | TransformerEncoderLayer | 1.3 M  | train
4 | global_forward_layer        | Linear                  | 128 K  | train
5 | forward_latent_mean_layer   | Linear                  | 131 K  | train
6 | forward_latent_logvar_layer | Linear                  | 131 K  | train
7 | soft                        | Softmax                 | 0      | train
8 | relu                        | ReLU                    | 0      | train
9 | fc1_dec                     | Linear                  | 31.5 K | train
--------------------------------------------------------------------------------
1.7 M     Tr

Training: |          | 0/? [00:00<?, ?it/s]

MisconfigurationException: ReduceLROnPlateau conditioned on metric val_loss which is not available. Available metrics are: ['train_loss', 'train_loss_step', 'train_loss_epoch', 'average_train_epoch_rec_loss', 'average_train_epoch_KL_loss']. Condition can be set using `monitor` key in lr scheduler dict